# Extract ObjectTables Results in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/21
- update : 2024/02/23


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.visualization import ZScaleInterval, LinearStretch, ImageNormalize


from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

from matplotlib.colors import ListedColormap

# LSST packages
from lsst.daf.butler import Butler
import lsst.geom as geom

# allow interactive plots
#%matplotlib widget
%matplotlib inline

# default plot style is accessible
plt.style.use('tableau-colorblind10')
import seaborn as sns

In [ ]:
# A repo which exists at the USDF, contains data from various instruments and
REPO = '/repo/embargo'

from lsst.daf.butler import Butler
from pprint import pprint

# Here we specify a default collection to use
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"
butler = Butler(REPO, collections=my_collection)
registry = butler.registry

In [ ]:
list(butler.registry.queryDimensionRecords('detector', where="instrument='LATISS' AND detector.id IN (0..8)"))

### access to skymap

https://github.com/sylvielsstfr/MyDP0LSS/blob/main/myDP0.2/TrackAndPatch/PlotTractAndPatchs_butler.ipynb

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

## Config

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType='skyMap', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
for i, ref in enumerate(datasetRefs):
    pprint(ref)
    skymap_id = ref.dataId["skymap"]
    if skymap_id ==  'latiss_v1':
        skymap = ref

In [ ]:
skymap = butler.get(ref)

In [ ]:
my_tracts = [ 5616,5634,5615,10644,383,8188,3863,455,452,5839,10643,382]

In [ ]:
#from lsst.daf.butler import Butler
#butler = Butler('SMALL_HSC')
skymap = butler.get('skyMap', skymap='latiss_v1', collections=my_collection)
tractInfo = skymap.generateTract(5616)
#patch = tractInfo[41]
#patch.getIndex()

In [ ]:
#for patch in tractInfo:
#    print(patch.getSequentialIndex())

In [ ]:
# Load data (deserialize)
import pickle
with open('../data/202402/Latiss_tractspatchs_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

In [ ]:
list_of_tracts = list(tracts_patchs_dict.keys())
list_of_tracts

In [ ]:
npatches = 0
for itract in list_of_tracts:
    npatches += len(tracts_patchs_dict[itract])
print(npatches)    

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

numTracts=0
# collection of polygons
patches = [] # container of all polygons


tract_polygon_dict = {}
tract_ra_dict = {}
tract_dec_dict = {}

# loop on tracts
#---------------
for tractInfo in skymap:
    tractID = tractInfo.getId()
    if tractID not in list_of_tracts:
        continue
    numTracts+=1  

    # container for each tract
    container_polygons_fortract = []
    container_ra_fortract = []
    container_dec_fortract = []


    # loop on patch
    #--------------
    for patch in tractInfo:
        patchID = patch.getSequentialIndex()
        if patchID not in tracts_patchs_dict[tractID]:
            continue
        ibb=patch.getInnerBBox()
        tWCS=tractInfo.getWcs()
        corners = []
        # loop on the 4 corners
        for icorn,corner in enumerate(ibb.getCorners()):
            p = geom.Point2D(corner.getX(), corner.getY())
            coord = tWCS.pixelToSky(p)
            corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
            container_ra_fortract.append(coord.getRa().asDegrees()) 
            container_dec_fortract.append(coord.getDec().asDegrees()) 
        #print(f"==={numTracts}==={tractID}:{patchID}==========")    
        corners_array = np.array(corners)
        corners_array = np.append(corners_array,[corners_array[0]],axis=0)
        polygon = Polygon(corners_array,closed=True)
        patches.append(polygon)
        container_polygons_fortract.append(polygon)
        
        
    tract_polygon_dict[tractID] = container_polygons_fortract
    tract_ra_dict[tractID] = np.array(container_ra_fortract)
    tract_dec_dict[tractID] = np.array(container_dec_fortract)
        
print(numTracts)   



## Plot all tracts together

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))

# loop on tracts
for itract in list_of_tracts:

   
    # extract the Polygons    
    patches = tract_polygon_dict[itract]

    # find color
    nPatches = len(tracts_patchs_dict[itract])
    sns.color_palette("hls", nPatches)
    my_cmap = ListedColormap(sns.color_palette("hls", nPatches))

    ra = tract_ra_dict[itract]
    dec = tract_dec_dict[itract]
    Xmin = ra.min()
    Xmax = ra.max()
    Ymin = dec.min()
    Ymax = dec.max()
    
    p = PatchCollection(patches, cmap=mpl.cm.jet, alpha=0.4)
    #p = PatchCollection(patches, cmap=my_cmap, alpha=1)

    colors = 50*np.random.rand(len(patches))
    p.set_array(np.array(colors))

    ax.add_collection(p)  

ax.set_xlim([0,300.])
ax.set_ylim([-80., 10.])

title = f"AUXTEL 2023-24 : tract {itract}, npatches{nPatches}"
ax.set_title(title)
ax.set_xlabel("ra")
ax.set_ylabel("dec")
#plt.gca().set_aspect('equal')
ax.axis('equal')
ax.grid()
plt.show()

## Plot tract One by One

In [ ]:
# loop on tracts
for itract in list_of_tracts:

   
    # extract the Polygons    
    patches = tract_polygon_dict[itract]

    # find color
    nPatches = len(tracts_patchs_dict[itract])
    sns.color_palette("hls", nPatches)
    my_cmap = ListedColormap(sns.color_palette("hls", nPatches))

    ra = tract_ra_dict[itract]
    dec = tract_dec_dict[itract]
    Xmin = ra.min()
    Xmax = ra.max()
    Ymin = dec.min()
    Ymax = dec.max()
    

    fig, ax = plt.subplots(figsize=(6,6))
    #p = PatchCollection(patches, cmap=mpl.cm.jet, alpha=0.4)
    p = PatchCollection(patches, cmap=my_cmap, alpha=1)

    #colors =   my_cmap.colors
    #p.set_array(np.array(colors))
    colors = 64*np.random.rand(len(patches))
    p.set_array(np.array(colors))

    ax.add_collection(p)  

    ax.set_xlim([Xmin,Xmax])
    ax.set_ylim([Ymin, Ymax])

    title = f"AUXTEL 2023-24 : tract {itract}, npatches{nPatches}"
    ax.set_title(title)
    ax.set_xlabel("ra")
    ax.set_ylabel("dec")
    #plt.gca().set_aspect('equal')
    ax.axis('equal')
    ax.grid()
    plt.show()